# Analyze

In [1]:
from google.colab import files
uploaded = files.upload()

import pandas as pd
df1 = pd.read_excel("correlationbetweenVariables.xlsx")
df1.head()

Saving correlationbetweenVariables.xlsx to correlationbetweenVariables.xlsx


,product_id,product_name,brand_name,price_usd,skin_tone,eye_color,skin_type,rating,is_recommended,helpfulness,review_text,review_title
0,379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0,lightMedium,green,combination,5,1.0,0.000000,I absolutely L-O-V-E this oil. I have acne pro...,A must have!
1,379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0,mediumTan,brown,oily,3,1.0,0.333333,I gave this 3 stars because it give me tiny li...,it keeps oily skin under control
2,379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0,lightMedium,brown,dry,5,1.0,1.000000,Works well as soon as I wash my face and pat d...,Worth the money!
3,379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0,fair,blue,combination,5,1.0,0.933333,Super short back story- when I was in my teens...,My favorite for many years
4,379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0,light,hazel,combination,5,1.0,1.000000,first oil that I‚Äôve tried and I really like ...,NaN


In [2]:
num_missing = df1.isna().sum()
num_missing

,0
product_id,0
product_name,0
brand_name,0
price_usd,0
skin_tone,22449
eye_color,20923
skin_type,11236
rating,0
is_recommended,12713
helpfulness,65462


In [3]:
df1[['is_recommended', 'helpfulness']] = df1[['is_recommended', 'helpfulness']].fillna(
    df1[['is_recommended', 'helpfulness']].mean()
)

In [4]:
import numpy as np

for col in ['skin_tone', 'eye_color', 'skin_type']:
    # Get value proportions (dropna ensures you only get observed values)
    probs = df1[col].value_counts(normalize=True, dropna=True)

    # How many values are missing?
    n_missing = df1[col].isna().sum()

    # Randomly sample values based on the observed distribution
    fill_values = np.random.choice(probs.index, size=n_missing, p=probs.values)

    # Fill in the missing values
    df1.loc[df1[col].isna(), col] = fill_values

In [5]:
df1.isnull().sum()

,0
product_id,0
product_name,0
brand_name,0
price_usd,0
skin_tone,0
eye_color,0
skin_type,0
rating,0
is_recommended,0
helpfulness,0


In [6]:
df1.describe()

,product_id,price_usd,rating,is_recommended,helpfulness
count,1.692940e+05,169294.000000,169294.000000,169294.000000,169294.000000
mean,5.858118e+05,57.596627,4.308097,0.845186,0.762803
std,3.522734e+06,53.740922,1.161649,0.347882,0.244294
min,2.046000e+03,3.000000,1.000000,0.000000,0.000000
25%,4.363682e+05,29.000000,4.000000,1.000000,0.762803
50%,4.671110e+05,44.000000,5.000000,1.000000,0.762803
75%,4.804470e+05,69.000000,5.000000,1.000000,1.000000
max,9.162736e+07,1900.000000,5.000000,1.000000,1.000000


In [36]:
search_term = input("Enter product ID or part of product name: ").lower()

Enter product ID or part of product name: 379064


In [46]:
def find_best_skin_type_per_product(df, product_query):
    product_query = str(product_query).strip()

    # Match by product ID or name
    if product_query.isdigit():
        matched_df = df[df['product_id'].astype(str) == product_query]
    else:
        matched_df = df[df['product_name'].str.lower().str.contains(product_query.lower())]

    if matched_df.empty:
        return f"No products found matching: {product_query}"

    result = []

    for product_id, group in matched_df.groupby('product_id'):
        product_name = group['product_name'].iloc[0]

        skin_stats = group.groupby('skin_type').agg(
            avg_rating=('rating', 'mean'),
            recommend_rate=('is_recommended', 'mean'),
            review_count=('rating', 'count')
        ).reset_index()

        skin_stats['score'] = 0.5 * skin_stats['avg_rating'] + 0.5 * skin_stats['recommend_rate'] * 5
        skin_stats = skin_stats[skin_stats['review_count'] >= 5]

        if skin_stats.empty:
            continue

        best_skin_type = skin_stats.sort_values('score', ascending=False).iloc[0]

        result.append({
            'product_id': product_id,
            'product_name': product_name,
            'best_skin_type': best_skin_type['skin_type'],
            'score': round(best_skin_type['score'], 2),
            'avg_rating': round(best_skin_type['avg_rating'], 2),
            'recommend_rate': round(best_skin_type['recommend_rate'], 2),
            'review_count': int(best_skin_type['review_count'])
        })

    if not result:
        return "No skin types had enough data to recommend."

    return pd.DataFrame(result).sort_values(by='score', ascending=False)

In [48]:
find_best_skin_type_per_product(df1, search_term)

,product_id,product_name,best_skin_type,score,avg_rating,recommend_rate,review_count
0,379064,Lotus Balancing & Hydrating Natural Face Treat...,normal,4.57,4.65,0.9,20
